![example](images/director_shot.jpeg)

# Project Title

**Authors:** Mitch Allison, Brenda De Leon, Matt Rubic
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import sqlite3

%matplotlib inline
conn = sqlite3.connect('./zippedData/im.db')

In [2]:
# loading the datasets
TheNumbers = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
RTinfo = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t')
RTreviews = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', encoding = 'unicode_escape')

### Rotten Tomatoes Movie Info dataset

In [3]:
RTinfo.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [4]:
RTinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


**Discovery**

May need to discover where the nulls are, convert string numbers to values. Looks like a lot of the Studio values are missing. Currency and box office seem to appear in the same entries, but only for like 20% of the data

**How this data might be used**

Could use this dataset to connect information about movies that appear in this dataset and others, to aggregate all information about a movie

### Rotten Tomatoes Reviews dataset

In [5]:
RTreviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [6]:
RTreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


**Discovery**

Rating is not currently a value, will need to be converted to become useful. Many ratings are NaN and will need to be dropped, and many other ratings are letters and will need to be dropped. ratings are sometimes on a different scale (read:denominator), but can be converted to real numbers.

Fresh or rotten seems to exist for each entry

Dates are also strings and need to be normalized to be used

ID be traced to which movie it represents, to aggregate

May want a new group for just reviews by top critics

**How this data might be used**

Could be used to display movie ratings alongside other information for common movies from other datasets

### The Numbers dataset

In [7]:
TheNumbers.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [8]:
TheNumbers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


**Discovery**

All entries in this dataset are non-null. However, all values are strings, so may need to be changed to be usable

**How this data might be used**

Use to evaluate and compare different financial information across movies, and aggregate with other information from different datasets

### IMDB database

In [9]:
query = '''
SELECT
    *
FROM
    movie_basics
--LIMIT 15
    '''

In [ ]:
movie_basics = pd.read_sql('''SELECT * ''', conn)
movie_basics

In [ ]:
movie_basics.info()

In [ ]:
query = '''
SELECT
    *
FROM
    movie_ratings
--LIMIT 15
    '''

In [ ]:
movie_ratings = pd.read_sql(query, conn)
movie_ratings

In [ ]:
movie_ratings.info()

In [ ]:
query = '''
SELECT
    *
FROM
    principals
--LIMIT 15
    '''

In [ ]:
principals = pd.read_sql(query, conn)
principals

In [ ]:
principals.info()

**Discovery**

This database contains a lot of data about movies that may help identify them, like id, title, release date, actors/directors, etc.

**How this data might be used**

It looks like this imdb database will be helpful for finding all movies from certain directors, or other such asks, but won't be as helpful for finding direct answers to our business questions.

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

**Cleaning data in TheNumbers**

TheNumbers represents our financial data we will use to analyze relationships between other variables and metrics derived from Gross revenue like profit, ROI, etc.

The following code converts dollar amounts to integer types within the database so they can be manipulated as numbers

In [ ]:
punct = '$,'   # `|` is not present here
money_to_num = str.maketrans(dict.fromkeys(punct, ''))

#stripping dollar-sign and commas from relevant objects
TheNumbers['worldwide_gross'] = '|'.join(TheNumbers['worldwide_gross'].tolist()).translate(money_to_num).split('|')
TheNumbers['domestic_gross'] = '|'.join(TheNumbers['domestic_gross'].tolist()).translate(money_to_num).split('|')
TheNumbers['production_budget'] = '|'.join(TheNumbers['production_budget'].tolist()).translate(money_to_num).split('|')
#converting number strings to int64 types
TheNumbers['worldwide_gross'] = TheNumbers['worldwide_gross'].astype(np.int64)
TheNumbers['domestic_gross'] = TheNumbers['domestic_gross'].astype(np.int64)
TheNumbers['production_budget'] = TheNumbers['production_budget'].astype(np.int64)

The following code creates metrics that measure Profit, Profit Margin, and ROI, given the information in TheNumbers dataset about Production Budget and Gross Domestic and Worldwide. We will use these metrics to evaluate and compare a film's profitability and the efficiency of an investment, which is more useful information than the Production Budget or Gross Revenue alone when we are seeking to explore factors that contribute to profitability. 

In [ ]:
#created variables for domestic profit, worldwide profit, domestic & worldwide profit margins (profit/gross)
#and domestic & worldwide ROIs (profit-production budget)
TheNumbers['domestic_profit'] = TheNumbers['domestic_gross'] - TheNumbers['production_budget']
TheNumbers['worldwide_profit'] = TheNumbers['worldwide_gross'] - TheNumbers['production_budget']
TheNumbers['domestic_margin'] = TheNumbers['domestic_profit'] / TheNumbers['domestic_gross']
TheNumbers['worldwide_margin'] = TheNumbers['worldwide_profit'] / TheNumbers['worldwide_gross']
TheNumbers['domestic_roi'] = TheNumbers['domestic_profit'] / TheNumbers['production_budget']
TheNumbers['worldwide_roi'] = TheNumbers['worldwide_profit'] / TheNumbers['production_budget']

The following code returns numbers rather than strings for release dates, so that this data can be more easily explored later 

In [ ]:
#Normalize release dates as numbers (original dataset had month names)
TheNumbers['release_date'] = pd.to_datetime(TheNumbers['release_date'])
#Creates new column in datset that approximates to calendar month and day as we seek to explore relationship 
#between calendar dates and revenue 
TheNumbers['calendar_day'] = TheNumbers['release_date'].astype(str).str.strip().str[-5:]
#Creates a function that assigns calendar day strings to a number. There is probably a simpler way to do this
def getdaynum(cal_day):
    day_list = []
    for entry in cal_day:
        if entry[:2] == '01':
            day_list.append(int(entry[3:]))
        elif entry[:2] == '02':
            day_list.append(int(entry[3:]) + 31)
        elif entry[:2] == '03':
            day_list.append(int(entry[3:]) + 60)
        elif entry[:2] == '04':
            day_list.append(int(entry[3:]) + 91)
        elif entry[:2] == '05':
            day_list.append(int(entry[3:]) + 122)
        elif entry[:2] == '06':
            day_list.append(int(entry[3:]) + 152)
        elif entry[:2] == '07':
            day_list.append(int(entry[3:]) + 182)
        elif entry[:2] == '08':
            day_list.append(int(entry[3:]) + 213)
        elif entry[:2] == '09':
            day_list.append(int(entry[3:]) + 244)
        elif entry[:2] == '10':
            day_list.append(int(entry[3:]) + 274)
        elif entry[:2] == '11':
            day_list.append(int(entry[3:]) + 305)
        elif entry[:2] == '12':
            day_list.append(int(entry[3:]) + 335)
    return pd.Series(day_list)
#creates new column in dataset for integer values of days
TheNumbers['release_day_num'] = getdaynum(TheNumbers['calendar_day'])

**Creating dataframe to analyze IMDB data with corresponding columns from TheNumbers**

The following code creates a dataframe that takes all of the production_budget info in from TheNumbers, and then is able to assign genres from IMDB. Connect the movie names to their genres in order to explore relationships between genre and financial performance. Groups the genres and chart vs total budget, profit, margins

In [ ]:
query = '''
SELECT
    primary_title,
    genres,
    start_year
FROM
    movie_basics
    '''
movie_genre = pd.read_sql(query, conn)

Clean genres into a list. Set movie name as index.

In [ ]:
movie_genre['genres_list'] = movie_genre['genres'].str.split()

Some movies have identical names. Make a new 'name_year' column to join on for both datasets.

In [ ]:
movie_genre['name_year'] = movie_genre['primary_title'] + movie_genre['start_year'].apply(str)
# TheNumbers['release_date'] is a datetime64[ns]
year = TheNumbers['release_date'].dt.year
TheNumbers['name_year'] = TheNumbers['movie']+ TheNumbers['release_date'].dt.strftime('%Y')
# Set name_year as index
movie_genre.set_index('name_year', inplace=True)
# Remove movies without genre
movie_genre.dropna(subset=['genres_list'], inplace=True)

Join movie_genre with TheNumbers

In [ ]:
money_genre = TheNumbers.join(movie_genre, on='name_year', how = 'inner')

**Some additional cleaning and outlier removal for genres**

Some movies had value 0 for domestic and worldwide gross, we chose to ignore them as they would skew data for movies that were shown in theaters

In [ ]:
money_genre = money_genre[(money_genre.domestic_gross != 0) & (money_genre.worldwide_gross != 0)]

Some movies are listed with multiple genres. We wanted to explore the the trends between a unique genre type and financial performance, so we got all unique genres and put them in a dictionary with the count of how often each appeared in the original dataset.

In [ ]:
genres = money_genre['genres'].unique().tolist()
unique_genres = []
for x in genres:
    templist = x.split(",")
    for y in templist:
        unique_genres.append(y)
unique_genres = list(set(unique_genres))
# genres_df is dict where key = genre, value = df of movies with that genre
genres_df = {x: money_genre[money_genre.genres.str.contains(x)] for x in unique_genres}

**Creating dataframes to analyze Actor and Director data with corresponding columns from TheNumbers**

Much like we did for genre, we want to explore relationships between actors and the financial performance of the movies they appear in, so we create and merge this dataframe

In [ ]:
# joined tables: movie_basics and persons to principals, selected only the matching records.
actors = pd.read_sql(
"""
SELECT
    category,
    pr.movie_id,
    pr.person_id,
    primary_name,
    primary_title 
FROM 
    principals AS pr
JOIN
    movie_basics AS mb using(movie_id)
JOIN
    persons AS pe using(person_id)
WHERE
    category = 'actor'
    or
    category = 'actress'
    or
    category = 'self'
""",conn)
actors

In [ ]:
# merging "actors" DataFrame (extracted from "IMDB") with "TheNumbers" DataFrame on 'primary_title' and 'movie'
actors_df = pd.merge(actors,
                  TheNumbers,
                  left_on='primary_title',
                  right_on='movie')

actors_df

In [ ]:
# joined tables: movie_basics and persons to principals, selected only the matching records.
directors = pd.read_sql(
"""
SELECT
    category,
    pr.movie_id,
    pr.person_id,
    primary_name,
    primary_title 
FROM 
    principals AS pr
JOIN
    movie_basics AS mb using(movie_id)
JOIN
    persons AS pe using(person_id)
WHERE
    category = 'director'
""",conn)
directors

In [ ]:
# merging "diretors" DataFrame (extracted from "IMDB") with "TheNumbers" DataFrame on 'primary_title' and 'movie'
directors_df = pd.merge(directors,
                  TheNumbers,
                  left_on='primary_title',
                  right_on='movie')

directors_df

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***